In [ ]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Set the desired figure size in inche
plt.rcParams['figure.figsize'] = (20, 12)  # Width: 8 inches, Height: 6 inches

#create my color array (viridis)
my_colors = ["#440154", "#3b528b", "#21918c",  "#fde725" ]


# add color array to set_palette
sns.set_theme(font_scale=1.6, style="ticks", palette= my_colors)


In [ ]:
# My file path
myDataPath = '..\\data\\AGE\\'

# This window is used to calculate the moving average of the data
window = 30

In [ ]:
import pandas as pd

def readWaterAge(FolderName):
    # Read the data from the CSV file into a DataFrame
    myVar = pd.read_csv(
        FolderName,
        sep=",",
        header=0,
        decimal=".",
        index_col="date and time",
        parse_dates=["date and time"]
    )
    
    # Rename columns
    myVar.rename(
        columns={"calculated  AGE  for water from source 1 (d)": "WaterAge"},
        inplace=True
    )
    
    # Rename the index
    myVar.index.names = ["Date"]
    
    # Filter rows to the desired date range
    myVar = myVar.loc["2021-12-20":"2022-12-31"].copy()
    
    # Drop any rows with NaN and convert WaterAge to float type
    myVar.dropna(subset=["WaterAge"], inplace=True)
    myVar["WaterAge"] = myVar["WaterAge"].astype(float)

    return myVar

In [ ]:
def smoothMyVar(var, myWindow):    
   # Calculate the rolling mean and standard deviation for the variable
    roll = var.WaterAge.rolling(myWindow, center=True)
    mySmoothValues = roll.agg(['mean', 'std']) 
    return mySmoothValues

In [ ]:
def date2deltaTime(var):
    # Calculate the time in years from using  the first data point
    ser_diff = var.index.to_series().diff()
    var['deltaT'] = ser_diff.dt.seconds.div(60, fill_value=0)
    var['deltaT'] = ((var['deltaT'].cumsum()))/(60*24*365)
    var.set_index('deltaT', inplace=True)
    return var   

In [ ]:
def plotSmootWaterAge(var, myWindow, myLabel):
    # Plot the rolling mean and standard deviation of the water age
    smoothValues = smoothMyVar(var, myWindow)
        
    ax = smoothValues['mean'].plot(linewidth=3, label= myLabel)            
    ax.fill_between(smoothValues.index, smoothValues['mean'] - smoothValues['std'], 
                    smoothValues['mean'] + smoothValues['std'], alpha=0.2)
          
    plt.tight_layout()
    plt.ylabel("Age trazer (days)", size = 28)
    plt.xlabel('Time (years)', size = 28)

    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    

    plt.legend() 
          
    return ax

In [ ]:
def plotTerminosAgesInlets(path, myWindow):
    # Plot the water age time series for the inlets and Carmen Island
    files_labels = [
        ('05_CarmenIsland.csv', 'Carmen Island'),
        ('01_PReal_inlet.csv', 'Puerto Real Inlet'),
        ('07_CarmenInlet.csv', 'Carmen Inlet') 

    ]
    
    # Loop through each file and label, read the water age, and plot
    for file, label in files_labels:
        water_age = readWaterAge(path + file)
        plotSmootWaterAge(water_age, myWindow, label)

# Plot the water age time series for the inlets and Carmen Island
plotTerminosAgesInlets(myDataPath, window)



In [ ]:
def plotTerminosAgesZones(path, myWindow):
    # Plot the water age time series for the zones
    files_labels = [

        ('02_Candelaria.csv', 'Candelaria Estuary'),
        ('03_Chumpan.csv', 'Chumpan Estuary'),
        ('04_lagoonCenter.csv', 'Lagoon Center'),
        ('06_Palizada.csv', 'Palizada Estuary')
    ]
    
    # Loop through each file and label, read the water age, and plot
    for file, label in files_labels:
        water_age = readWaterAge(path + file)
        plotSmootWaterAge(water_age, myWindow, label)

# Plot the water age time series for the interes zones
plotTerminosAgesZones(myDataPath, window)


In [ ]:
def plotTerminosAgesSubplots(path, myWindow):
    # Plot the water age time series for the inlets and zones in subplots
    fig, axs = plt.subplots(2, 1, figsize=(15, 20))
    
    # Plot inlets on the top subplot
    plt.sca(axs[0])
    plotTerminosAgesInlets(path, myWindow)

    
    # Plot zones on the bottom subplot
    plt.sca(axs[1])
    plotTerminosAgesZones(path, myWindow)

    plt.tight_layout()
    
    plt.savefig('..\output_files\\water_age_zones.pdf', format='pdf')
    
    plt.show()

plotTerminosAgesSubplots(myDataPath, window)

In [ ]:
import numpy as np
from tabulate import tabulate

def calculate_mean_std_print_table(path):
    # Calcutae the mean and standard deviation of the water age for each file and print in a table format

    # Define file names and corresponding labels
    files_labels = [ 
        ('07_CarmenInlet.csv', 'Carmen Inlet'),
        ('01_PReal_inlet.csv', 'Puerto Real Inlet'),
        ('05_CarmenIsland.csv', 'Carmen Island'),
        ('06_Palizada.csv', 'Palizada Estuary'),
        ('04_lagoonCenter.csv', 'Lagoon Center'),
        ('03_Chumpan.csv', 'Chumpan Estuary'),
        ('02_Candelaria.csv', 'Candelaria Estuary')
    ]
   
    # Read all files and store in a dictionary
    water_ages = {label: readWaterAge(path + file) for file, label in files_labels}

    # Calculate mean and standard deviation for each file
    stats = {label: (np.mean(water_age).item(), np.std(water_age, axis=0).item()) for label, water_age in water_ages.items()}


    #Prepare data for tabulate
    table_data = [[label, f"{mean:.0f} ± {std:.0f}"] for label, (mean, std) in stats.items()]
    
    # Print mean and standard deviation in a table format using tabulate
    print(tabulate(table_data, headers=["Zones", "Mean ± Standard Deviationn (days)"], tablefmt="grid"))

# Print the mean and standard deviation of the water age for each file
calculate_mean_std_print_table(myDataPath)    